# QUBO Portfolio Optimisation for a Single Football Match

This notebook demonstrates a **portfolio optimisation engine for sportsbook bets** using a QUBO / Ising formulation and the ORBIT probabilistic simulator.

Our solution focuses on **optimising a portfolio of sportsbook bets according to the user’s risk appetite**, rather than searching for strict risk-free arbitrage.

- Each bet is treated as a **node** in a graph, with a weight equal to its **expected value (EV)** based on our probability estimate and the market odds.
- **Edges** represent **correlation or overlap between bets** – for example, multiple bets depending on the same match outcome or scoreline.
- The optimisation goal is to select a subset of bets that **maximises total positive alpha** while penalising portfolios that are **over-exposed to the same risks**.

In other words, we help a bettor (or trading desk) allocate their bankroll intelligently:  
**focus on value, avoid redundant risk, and respect a chosen risk appetite.**

Although we demonstrate the method on sports betting, the underlying optimisation framework is general.  
Treating positions as nodes, their relationships as edges, and minimising an energy function to pick a portfolio can be applied in other financial settings too – for example **options selection**, **trades within the same asset class**, or **small diversified portfolios** where interactions matter.

---

## Structure of this notebook

1. **Data preparation**  
   - Load and clean a set of markets for a single football match.  
   - Build a time series of odds and implied probabilities for multiple markets.

2. **QUBO formulation (portfolio objective)**  
   - Compute the **expected value (EV)** of each bet.  
   - Build a **QUBO energy function** where:  
     - linear terms encode the value of including each bet;  
     - quadratic terms encode correlation / overlap penalties between bets.

3. **Brute-force validation (small N)**  
   - For a modest number of markets (e.g. N ≈ 20), exhaustively evaluate all 2^N portfolios.  
   - Use this to obtain the exact ground state and validate the quality of ORBIT’s solution.

4. **QUBO → Ising mapping and ORBIT optimisation**  
   - Map the QUBO to an Ising model with couplings J and fields h.  
   - Run the ORBIT simulator to approximately minimise the Ising energy.  
   - Decode the resulting spin configuration back into a set of bets.

5. **Comparison and interpretation**  
   - Compare ORBIT’s solution to the brute-force optimum and simulated-annealing algorithm
   - Inspect the selected bets, the number of positions, and the implied “quality” of the portfolio.

The emphasis is not on speed or large-scale deployment in this notebook, but on a **clear, end-to-end demonstration** of how a realistic optimisation problem is encoded as a QUBO and solved using probabilistic computing.


In [1]:
import numpy as np
import pandas as pd
import itertools
import time
import orbit 
from pathlib import Path

In [2]:
#loading the data in - add the file path into csv_path
csv_path = Path('/Users/ehsanmassah/Documents/Quantum Dice/team_entropica_repo/probabilistic-computing-arbitrage/odds_long.csv')
df = pd.read_csv(csv_path)

required_cols = {'timestamp', 'market', 'odds', 'implied_prob'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

df['timestamp'] = pd.to_datetime(df['timestamp'])
odds_long = df.copy()

print("Loaded odds data from CSV:")
odds_long.head()

Loaded odds data from CSV:


,timestamp,market,odds,implied_prob
0,2025-11-28 05:00:00+00:00,money_line_away,4.020,0.248756
1,2025-11-28 05:00:00+00:00,money_line_draw,4.130,0.242131
2,2025-11-28 05:00:00+00:00,money_line_home,1.740,0.574713
3,2025-11-28 05:00:00+00:00,spread_-0.25_away,2.460,0.406504
4,2025-11-28 05:00:00+00:00,spread_-0.25_home,1.543,0.648088


## QUBO formulation: betting portfolio as a binary optimisation problem

We model the selection of bets as a **binary vector**:

- Let \( x_i \in \{0,1\} \) indicate whether bet \( i \) is included in the portfolio.  
- For each bet, we define an **expected value per unit stake**:
  \[
  \text{EV}_i = p_i \cdot \text{odds}_i - 1,
  \]
  where \( p_i \) is our (subjective or model-based) probability for that outcome and \( \text{odds}_i \) is the decimal odds.

We then construct a **QUBO energy function** of the form:
\[
E_{\text{QUBO}}(x) = \sum_i w_i x_i + \sum_{i<j} Q_{ij} x_i x_j.
\]

In this notebook:

- The **linear term** \( w_i \) is taken as
  \[
  w_i = -\text{EV}_i,
  \]
  so that bets with higher expected value contribute **lower energy** when selected (since we minimise \( E \)).  
- The **quadratic term** \( Q_{ij} \) encodes **correlations or overlaps** between bets:
  - If two bets load on the same underlying match or outcome, a positive \( Q_{ij} \) penalises taking them together.
  - If two bets are relatively independent, \( Q_{ij} \) will be small.

The overall effect is:

- Portfolios with **many high-EV bets** have **low linear energy** (good).  
- Portfolios that are **over-exposed to the same outcome** incur **quadratic penalties** (bad).  

By minimising \( E_{\text{QUBO}}(x) \), we search for a portfolio that balances **value (alpha)** and **risk concentration**, consistent with the user’s risk appetite.


In [3]:
def expected_value(p, odds):
    """
    Expected profit per unit stake:
    EV = p * odds - 1
    """
    return p * odds - 1.0

def portfolio_energy_qubo(x, w, Q):
    """
    QUBO energy:
    E(x) = sum_i w_i x_i + sum_{i<j} Q_ij x_i x_j
    x : 1D numpy array of 0/1 of length N
    """
    linear = np.dot(w, x)
    quadratic = 0.0
    N = len(x)
    for i in range(N):
        for j in range(i+1, N):
            quadratic += Q[i, j] * x[i] * x[j]
    return linear + quadratic

In [4]:
def compute_market_correlation(df, value_col="implied_prob"):
    """
    Compute the correlation matrix of N betting markets based on
    implied probabilities (or odds).

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns ['timestamp', 'market', value_col]
    value_col : str
        Column name to compute correlation on ("implied_prob" or "odds")

    Returns
    -------
    corr_matrix : pd.DataFrame
        N×N correlation matrix of markets.
    wide_df : pd.DataFrame
        Time-indexed wide DataFrame used for correlation.
    """

    wide_df = df.pivot_table(
        index="timestamp",
        columns="market",
        values=value_col
    )
    corr_matrix = wide_df.corr()
    return corr_matrix, wide_df

if __name__ == "__main__":
    corr, wide = compute_market_correlation(df)
    print("\n=== Correlation Matrix (20×20) ===")
    print(corr)


=== Correlation Matrix (20×20) ===
market             money_line_away  money_line_draw  money_line_home  \
market                                                                 
money_line_away           1.000000         0.479996        -0.703479   
money_line_draw           0.479996         1.000000        -0.205773   
money_line_home          -0.703479        -0.205773         1.000000   
spread_-0.25_away         0.789939         0.384678        -0.804846   
spread_-0.25_home        -0.718123        -0.189653         0.888489   
spread_-0.5_away          0.752246         0.388055        -0.669951   
spread_-0.5_home         -0.710478        -0.248891         0.825802   
spread_-0.75_away         0.820436         0.408864        -0.829860   
spread_-0.75_home        -0.724798        -0.194968         0.899278   
spread_-1.0_away          0.818199         0.665896        -0.686144   
spread_-1.0_home         -0.760536        -0.553249         0.618622   
spread_-1.25_away         0.

In [6]:
#Taking the lastest odds from the API
final_snapshot = (
    df.sort_values("timestamp")
      .groupby("market")
      .tail(1)
      .set_index("market")
)
final_snapshot

,timestamp,odds,implied_prob
market,,,
spread_-0.5_away,2025-12-02 22:00:00+00:00,2.250,0.444444
spread_-0.25_home,2025-12-03 07:00:00+00:00,1.512,0.661376
spread_0.0_away,2025-12-03 08:00:00+00:00,3.510,0.284900
spread_-0.5_home,2025-12-03 08:00:00+00:00,1.694,0.590319
money_line_home,2025-12-03 10:00:00+00:00,1.694,0.590319
spread_-0.75_home,2025-12-03 11:00:00+00:00,1.892,0.528541
spread_-0.25_away,2025-12-03 11:00:00+00:00,2.680,0.373134
spread_-0.75_away,2025-12-03 12:00:00+00:00,2.030,0.492611
spread_0.25_away,2025-12-04 03:00:00+00:00,3.950,0.253165


In [7]:
p_final = final_snapshot["implied_prob"].values
odds_final = final_snapshot["odds"].values

# Expected value per market
EV = p_final * odds_final - 1.0

# Linear QUBO term
w = -EV

In [8]:
wide = df.pivot_table(
    index="timestamp",
    columns="market",
    values="implied_prob"
).sort_index()

rho = wide.corr().values  # 20 × 20 matrix
Q = rho

In [9]:
#Brute-Force Calculations
N = len(final_snapshot)
all_results = []

start_time = time.perf_counter() 

for bits in itertools.product([0, 1], repeat=N):
    x = np.array(bits)
    E = -1 * portfolio_energy_qubo(x, w, Q)
    all_results.append({
        "x": bits,
        "E": E,
        "num_bets": x.sum()
    })

end_time = time.perf_counter() 
elapsed_bf = end_time - start_time

results_df = (
    pd.DataFrame(all_results)
      .sort_values("E", ascending=True)
      .reset_index(drop=True)
)

print(results_df.head(32))
print(f"\nChecked {2**N} Configurations for N={N}.")
print(f"Brute Force - Time to Ground State: {elapsed_bf:.4f} seconds")

                                                    x          E  num_bets
0   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -39.744933        11
1   (1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -34.421108        10
2   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -33.298907        12
3   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -33.255328        10
4   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -33.009453        12
5   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, ... -32.760380        12
6   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ... -32.738757        12
7   (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -32.694881        12
8   (1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -32.651786        12
9   (1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -32.635863        12
10  (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ... -32.635204        12
11  (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... -32.598557        10
12  (1, 1, 0, 1, 0, 1, 0,

## QUBO → Ising mapping and interpretation of scaling factors

The ORBIT simulator expects an **Ising model** of the form
\[
E_{\text{Ising}}(s) = \sum_i h_i s_i + \sum_{i<j} J_{ij} s_i s_j,
\]
where each spin variable \\( s_i \{-1, +1\} \\).

Our QUBO is defined in terms of binary variables \\( x_i \in \{0,1\} \\):
\[
E_{\text{QUBO}}(x) = \sum_i w_i x_i + \sum_{i<j} Q_{ij} x_i x_j.
\]

The standard mapping between these two formulations uses
\[
x_i = \frac{1 + s_i}{2}.
\]

If we substitute \\( x_i = (1+s_i)/2 \\) into the QUBO and collect terms, we obtain an Ising model with:

- couplings
  \[
  J_{ij} = \frac{Q_{ij}}{4},
  \]
- local fields
  \[
  h_i = \frac{w_i}{2} + \frac{1}{4}\sum_{j \neq i} Q_{ij},
  \]
- plus an additive constant energy shift that does not affect which configuration minimises the energy.

This explains the scaling factors that appear in the code:

- `J0 = Q / 4.0`  
- `h0[i] = w[i] / 2.0 + 0.25 * np.sum(Q[i, :])`

Two important points for interpretation:

1. **Absolute energies differ, minimisers do not**  
   The Ising energy and the original QUBO energy can have different absolute values because of constant shifts and rescaling. What matters is that they share the **same minimising configuration**.

2. **Validation via brute force**  
   For the small problem size in this notebook, we perform an exhaustive search over all \\( 2^N \\) portfolios to find the exact ground state of the QUBO.  
   We then run ORBIT on the corresponding Ising instance and **compare ORBIT’s solution to the brute-force optimum**, confirming that the mapping and implementation are consistent.


In [22]:
#Orbit Calculations
N = len(final_snapshot)
bets = final_snapshot.index.tolist()  

# Standard QUBO -> Ising mapping 
J0 = Q / 4.0
h0 = np.zeros(N)
for i in range(N):
    h0[i] = w[i] / 2.0 + 0.25 * np.sum(Q[i, :])

ising_J = J0
ising_h = h0

start_time = time.perf_counter()
result = orbit.optimize_ising(
    -1*ising_J,
    -1*ising_h,
    n_replicas=5,
    full_sweeps=50_000,
    beta_initial=0.35,
    beta_end=3.5,
    beta_step_interval=1,
)

elapsed_orb = time.perf_counter() - start_time

s_star = np.array(result.min_state)  
x_star = (1 + s_star) // 2                 

E_orbit = -1 * portfolio_energy_qubo(x_star, w, Q)

chosen_bets = [b for b, bit in zip(bets, x_star) if bit == 1]

print("=== ORBIT optimisation result ===")
print(f"Time to (approximately) find ground state: {elapsed_orb:.4f} seconds\n")

print("Spins (s*):", s_star.tolist())
print("Bits  (x*):", x_star.tolist())
print("Number of bets in portfolio:", int(x_star.sum()))
print("Selected bets:", chosen_bets)

print("\nORBIT reported min_cost:", result.min_cost)
print("Objective E = -portfolio_energy_qubo(x*, w, Q):", E_orbit)

# Optional: compare with brute-force ground state if you still have results_df
if 'results_df' in globals():
    print("\n=== Comparison with brute force ===")
    print("Brute-force best E:", results_df.loc[0, 'E'])
    print("Brute-force best x:", results_df.loc[0, 'x'])

[2025-12-06 20:37:11] INFO - orbit.simulator: Simulation starting...
[2025-12-06 20:37:56] INFO - orbit.simulator: Simulation completed in 42.65 seconds
=== ORBIT optimisation result ===
Time to (approximately) find ground state: 84.4195 seconds

Spins (s*): [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
Bits  (x*): [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
Number of bets in portfolio: 10
Selected bets: ['spread_-0.5_away', 'spread_-0.25_home', 'spread_-0.5_home', 'spread_-0.75_home', 'spread_-0.75_away', 'spread_0.25_home', 'money_line_away', 'spread_-1.0_away', 'money_line_draw', 'spread_-1.75_home']

ORBIT reported min_cost: -67.64294251688032
Objective E = -portfolio_energy_qubo(x*, w, Q): -33.255327824591035

=== Comparison with brute force ===
Brute-force best E: -39.74493253922423
Brute-force best x: (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)


In [23]:
# --- Check ORBIT solution against brute-force results and rank it ---
orbit_bits = tuple(int(b) for b in x_star)
matches = results_df[results_df["x"] == orbit_bits]

if matches.empty:
    print("⚠ ORBIT bitstring not found in brute-force results (this should not happen if N matches).")
else:
    match_idx = matches.index[0]   # 0-based index in results_df
    rank = match_idx + 1           # human-friendly rank (1 = best)

    E_best = results_df.loc[0, "E"]
    x_best = results_df.loc[0, "x"]
    E_orbit_bruteforce = matches.iloc[0]["E"]

    print("\n=== ORBIT vs Brute Force ===")
    print(f"ORBIT bitstring: {orbit_bits}")
    print(f"ORBIT energy from brute-force table: {E_orbit_bruteforce:.6f}")
    print(f"Ground state energy (brute force):   {E_best:.6f}")
    print(f"Ground state bitstring:              {x_best}")

    print(f"\nRank of ORBIT state among all 2^{N} configs: {rank} (1 = ground state)")
    print(f"Energy gap to ground state: {E_orbit_bruteforce - E_best:.6f}")

    # Optional: how many configs are strictly better / equal
    n_better = (results_df["E"] < E_orbit_bruteforce).sum()
    n_equal  = (results_df["E"] == E_orbit_bruteforce).sum()
    print("")
    print(f"Time to the lowest state using orbit: {elapsed_orb:.4f} seconds\n")
    print(f"Time to find the ground state using brute-force  : {elapsed_bf:.4f} seconds\n")


=== ORBIT vs Brute Force ===
ORBIT bitstring: (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0)
ORBIT energy from brute-force table: -33.255328
Ground state energy (brute force):   -39.744933
Ground state bitstring:              (1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)

Rank of ORBIT state among all 2^21 configs: 4 (1 = ground state)
Energy gap to ground state: 6.489605

Time to the lowest state using orbit: 84.4195 seconds

Time to find the ground state using brute-force  : 155.1704 seconds



In [19]:
def sa_optimize_qubo(w, Q, n_steps=100_000, T_start=5.0, T_end=0.01, seed=123):
    rng = np.random.default_rng(seed)
    N = len(w)

    def energy(x):
        return -portfolio_energy_qubo(x, w, Q)  # same sign convention as brute-force

    x = rng.integers(0, 2, size=N, dtype=int)
    E = energy(x)

    best_x = x.copy()
    best_E = E

    for step in range(n_steps):
        T = T_start * (T_end / T_start) ** (step / max(1, n_steps - 1))

        i = rng.integers(0, N)
        x_new = x.copy()
        x_new[i] = 1 - x_new[i]

        E_new = energy(x_new)
        dE = E_new - E

        if dE <= 0 or rng.random() < np.exp(-dE / T):
            x, E = x_new, E_new
            if E < best_E:
                best_E = E
                best_x = x.copy()

    return best_x, best_E

start = time.perf_counter()
x_sa, E_sa = sa_optimize_qubo(w, Q, n_steps=100_000)
elapsed_sa = time.perf_counter() - start

print("=== Simulated Annealing result ===")
print("Bits (x*):", x_sa.tolist())
print("Number of bets in portfolio:", int(x_sa.sum()))
print(f"SA energy: {E_sa:.6f}")
print(f"Time to (approximately) find ground state: {elapsed_sa:.4f} seconds")

if "results_df" in globals():
    sa_bits = tuple(int(b) for b in x_sa)
    matches = results_df[results_df["x"] == sa_bits]

    if matches.empty:
        print("\nSA bitstring not found in brute-force table.")
    else:
        idx = matches.index[0]
        rank = idx + 1
        E_best = results_df.loc[0, "E"]
        print("\n=== SA vs Brute Force ===")
        print(f"Rank of SA state among all 2^N configs: {rank} (1 = ground state)")
        print(f"Ground state energy (brute force): {E_best:.6f}")
        print(f"Energy gap to ground state: {E_sa - E_best:.6f}")

=== Simulated Annealing result ===
Bits (x*): [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
Number of bets in portfolio: 11
SA energy: -39.744933
Time to (approximately) find ground state: 7.3363 seconds

=== SA vs Brute Force ===
Rank of SA state among all 2^N configs: 1 (1 = ground state)
Ground state energy (brute force): -39.744933
Energy gap to ground state: 0.000000


## Summary and typical usage

In this notebook we have:

- Built a realistic **sportsbook portfolio optimisation** instance for a single football match.  
- Encoded the problem as a **QUBO** where:
  - linear terms reward high expected value bets,  
  - quadratic terms penalise correlated / overlapping exposures.  
- Exhaustively searched all \(2^N\) portfolios for a modest-sized universe (N ≈ 20) to obtain the **exact ground state**.
- Mapped the QUBO to an **Ising model** and used **ORBIT** to approximately minimise the Ising energy.
- Compared the ORBIT solution to the brute-force optimum and inspected the selected portfolio of bets.

For a typical user with more compute, the same pipeline can be applied to **larger sets of markets**:

- The data feed would come from live sportsbook APIs rather than static files.  
- The correlation structure \(Q\) can incorporate more sophisticated measures of dependence between markets.  
- ORBIT can be used to explore the energy landscape and propose **high-quality portfolios** within the user’s risk and bankroll constraints.

This notebook is therefore intended as an **end-to-end, validated proof of concept** that a real combinatorial portfolio problem in sports betting can be:

1. Formulated as a QUBO,
2. Mapped to an Ising model, and
3. Solved using probabilistic computing techniques such as ORBIT.
